In [7]:
import math

def emo_lev(arousal,valence):    #Function that returns emotion and intensity
    arousal -= 5
    valence -= 5
    r = 5
    x, y = valence, arousal
    dist = (x**2+y**2)**0.5

    #Calc theta in radian
    theta = math.atan(y/x)

    #Convert theta from rad to deg
    theta = 180 * theta/math.pi

    def lev(dist):
        if(dist<1.67):
            return "Low"
        if(dist>1.67 and dist<3.34):
            return "Medium"
        else:
            return "High"
        
    #Check which intensity
    levl = lev(dist)


    def conv_emo(theta):
        if(x>0 and y>0):  #1st Quadrant
            if theta <= 90:
                if(theta<45):
                    return "Happy",1.1,theta  #1st Quad, 1st half
                else:
                    return "Excited",1.2,theta #1st Quad, 2nd half
        else:
            if(x>0 and y<0): #4
                theta+=360
                quad = 4
                if theta <= 360:
                    if(theta<315):
                        return "Calm",4.1,theta
                    else:
                        return "Content",4.2,theta
            else:
                if(x<0 and y>0): #2
                    theta+=180
                    quad = 2
                    if theta <= 180:
                        if(theta<135):
                            return "Angry",2.1,theta
                        else:
                            return "Afraid",2.2,theta
                else:
                    if(x<0 and y<0):
                       theta+=180
                       if theta <= 270:
                            if(theta<225):
                                return "Sad",3.1,theta
                            else:
                                return "Depressed",3.2,theta
                       
                   
    #print("Theta before conversion: {}".format(theta))
    emo,quad,theta = conv_emo(theta)
    #print("Distance = {} \nTheta after conversion = {} \nquadrant = {} ".format(dist,theta,quad))
    return emo,levl,theta,quad    #END of emo_lev

#Input the valence, arousal values here.
valence_x, arousal_y = 2,3
emotion, level, theta,quadrant = emo_lev(arousal_y,valence_x)

#Optional printing. Just return the values you need if you want.
print("Emotion    : "+emotion)
print("Intensity  : "+level)
print("Theta      : {} degree".format("%.2f"%theta))
print("Quadrant   : {}".format(quadrant))

Emotion    : Sad
Intensity  : High
Theta      : 213.69 degree
Quadrant   : 3.1


In [8]:
from models.lstm import EEGLSTM
from dataset.DEAP_DATASET import ModularDeapDataset
from torch.utils.data import DataLoader
import torch
from util.train import select_last_elm

DATA_SET_PATH = 'dataset/'
deap_val = ModularDeapDataset(DATA_SET_PATH, train=False)
loader = DataLoader(deap_val, shuffle=True, batch_size=1)

INPUT_SIZE = 40
HIDDEN_SIZE1 = 64
HIDDEN_SIZE2 = 32
OUTPUT_SIZE = 4

MDL_PATH = 'models/saved_weights/lstm_v2_dropout.pth'

model = EEGLSTM(HIDDEN_SIZE1, HIDDEN_SIZE2, 1)

model.load_state_dict(torch.load(MDL_PATH, map_location="cpu"))
model.eval()

with torch.no_grad():
    for i, (signal, label) in enumerate(loader):
        hidden = model.initHidden()
        output = model(signal, hidden)
        output = select_last_elm(output, "cpu")
        output = output.numpy()
        label = label.numpy()
        print("Predicted:", output, "Actual Label:", label)
        
        emotion, level, theta,quadrant = emo_lev(output[0][1],output[0][0])
        print("PREDICTED",output)
        print("\tEmotion    : "+emotion)
        print("\tIntensity  : "+level)
        print("\tTheta      : {} degree".format("%.2f"%theta))
        print("\tQuadrant   : {}".format(quadrant))
        
        emotion, level, theta,quadrant = emo_lev(label[0][1],label[0][0])
        print("ACTUAL",label)
        print("\tEmotion    : "+emotion)
        print("\tIntensity  : "+level)
        print("\tTheta      : {} degree".format("%.2f"%theta))
        print("\tQuadrant   : {}".format(quadrant))


Predicted: [[5.5199795 5.355178  5.804351  6.0802584]] Actual Label: [[3.87 7.15 7.13 6.23]]
PREDICTED
	Emotion    : Happy
	Intensity  : Low
	Theta      : 34.34 degree
	Quadrant   : 1.1
ACTUAL
	Emotion    : Angry
	Intensity  : Medium
	Theta      : 117.73 degree
	Quadrant   : 2.1
Predicted: [[4.4810624 4.7715697 4.877444  4.6598945]] Actual Label: [[5.15 3.   5.04 2.01]]
PREDICTED
	Emotion    : Sad
	Intensity  : Low
	Theta      : 203.76 degree
	Quadrant   : 3.1
ACTUAL
	Emotion    : Calm
	Intensity  : Medium
	Theta      : 274.29 degree
	Quadrant   : 4.1
Predicted: [[5.7121444 5.3038316 6.5192766 5.5020165]] Actual Label: [[7.15 5.94 9.   4.81]]
PREDICTED
	Emotion    : Happy
	Intensity  : Low
	Theta      : 23.11 degree
	Quadrant   : 1.1
ACTUAL
	Emotion    : Happy
	Intensity  : Medium
	Theta      : 23.62 degree
	Quadrant   : 1.1
Predicted: [[3.6790686 4.6709294 4.3347907 5.1179852]] Actual Label: [[8.09 7.04 6.   9.  ]]
PREDICTED
	Emotion    : Sad
	Intensity  : Low
	Theta      : 193.99 deg

KeyboardInterrupt: 